In [6]:
import pandas as pd
import requests

# URL with timestamp range: Jan 7, 2023 – Present (UNIX timestamps)
url = "https://finance.yahoo.com/quote/RELIANCE.BO/history?period1=1673049600&period2=1744065736"

# Add headers to mimic a browser request
headers = {
	'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

try:
	# Get the page content with headers
	response = requests.get(url, headers=headers)
	response.raise_for_status()  # Raise exception for HTTP errors
	
	# Load all tables from the page
	tables = pd.read_html(response.text)
	
	# Usually the first table is the one we want
	df = tables[0]
	
	# Find close column name (it might vary between 'Close*' or something else)
	close_col = [col for col in df.columns if 'Close' in col and 'Adj' not in col][0]
	
	# Remove rows like 'Dividend' or empty rows
	df = df[~df[close_col].astype(str).str.contains('Dividend', na=False)]
	
	# Rename columns to simpler names
	df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj_Close', 'Volume']
	
	# Convert data types
	df['Date'] = pd.to_datetime(df['Date'])
	numeric_cols = ['Open', 'High', 'Low', 'Close', 'Adj_Close']
	df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
	df['Volume'] = pd.to_numeric(df['Volume'], errors='coerce').astype('Int64')
	
	# Save to CSV
	df.to_csv("reliance_stock_data.csv", index=False)
	
	print("CSV saved as reliance_stock_data.csv")
	print(f"Number of rows in the data: {len(df)}")
except Exception as e:
	print(f"An error occurred: {e}")


CSV saved as reliance_stock_data.csv
Number of rows in the data: 552


C:\Users\sagar\AppData\Local\Temp\ipykernel_12648\113909241.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)
C:\Users\sagar\AppData\Local\Temp\ipykernel_12648\113909241.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df['Date'])
C:\Users\sagar\AppData\Local\Temp\ipykernel_12648\113909241.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu